# Soft Computing

## Vežba 3 - HOG

## Histogram of Oriented Gradients (HOG)

Jedan od najpopularnijih postupaka za izdvajanje osobina (eng. *feature extraction*) sa digitalne slike. Formalno, HOG je nešto što se zove **deskriptor slike** (eng. *image descriptor*), koji predstavlja opis vizuelnih osobina slike. Iako je predstavljen pre više od jedne dekade, tačnije 2005. godine u radu <a href="http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf">Histogram of Oriented Gradients for Human Detection</a>, HOG je deskriptor koji se i dan-danas vrlo često koristi za probleme detekcije i prepoznavanja objekata na slikama.

<img src="images/hog_diagram.png">

## Support Vector Machine (SVM)

SVM je model nadgledanog mašinskog učenja, koji se može koristiti za probleme klasifikacije i regresije. Ugrubo, SVM klasifikator konstruiše hiperravan koja "najbolje" razdvaja klase podataka, a ova "najbolja" hiperravan je zapravo ona koja maksimizuje marginu između podataka različitih klasa.

U svom osnovnom obliku, SVM je linearni klasifikator, tj. može vrlo efikasno da pronađe hiperravan koja maksimizuje marginu između linearno separabilnih podataka. Međutim, korišćenjem tzv. kernel trika (eng. *kernel trick*), moguće je konstruisati SVM za klasifikaciju podataka koji nisu linearno separabilni - tačnije, pomoću kernela se ulazni podaci zapravo mapiraju na neki višedimenzionalni prostor gde ti podaci postanu linearno separabilni. 

<img src="images/svm.png">

## Detekcija automobila

HOG i SVM ćemo iskoristiti kako bi rešili problem detekcije automobila na slikama iz <a href="http://cogcomp.org/Data/Car/">UIUC Image Database for Car Detection</a> skupa podataka. 

Trening skup **data/CarData/TrainImages** sadrži 1050 slika (pozitivnih i negativnih primera) dimenzija 100 x 40 za obučavanje i validaciju deskriptora i klasifikatora.

Test skup **data/CarData/TestImages** sadrži 170 slika različitih dimenzija za evaluaciju rada deskriptora i klasifikatora.

Rezultati rada deskriptora i klasifikatora na testnom skupu podataka se mogu uporediti sa tačnim lokacijama automobila koji se nalaze u **data/CarData/trueLocations.txt**.

### Obučavanje

In [ ]:
import os
import numpy as np
import cv2 # OpenCV
from sklearn.svm import SVC # SVM klasifikator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier # KNN
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)

def display_image(image):
    plt.imshow(image, 'gray')

#### Primer pozitivnog uzorka - jeste automobil

In [ ]:
ipos = load_image('data/CarData/TrainImages/pos-0.pgm')
display_image(ipos)

In [ ]:
ipos.shape

##### Primer negativnog uzorka - nije automobil

In [ ]:
ineg = load_image('data/CarData/TrainImages/neg-0.pgm')
display_image(ineg)

In [ ]:
ineg.shape

#### Učitavanje svih primera

In [ ]:
train_dir = 'data/CarData/TrainImages/'

pos_imgs = []
neg_imgs = []

for img_name in os.listdir(train_dir):
    img_path = os.path.join(train_dir, img_name)
    img = load_image(img_path)
    if 'pos' in img_name:
        pos_imgs.append(img)
    elif 'neg' in img_name:
        neg_imgs.append(img)
        
print("Positive images #: ", len(pos_imgs))
print("Negative images #: ", len(neg_imgs))

#### Izračunavanje HOG deskriptora za sve slike i formiranje niza labela (1 = jeste automobil, 0 = nije automobil)

In [ ]:
pos_features = []
neg_features = []
labels = []

nbins = 9 # broj binova
cell_size = (8, 8) # broj piksela po celiji
block_size = (3, 3) # broj celija po bloku

hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1], 
                                  img.shape[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

for img in pos_imgs:
    pos_features.append(hog.compute(img))
    labels.append(1)

for img in neg_imgs:
    neg_features.append(hog.compute(img))
    labels.append(0)

pos_features = np.array(pos_features)
neg_features = np.array(neg_features)
x = np.vstack((pos_features, neg_features))
y = np.array(labels)

#### Podela trening skupa na trening i validacioni 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print('Train shape: ', x_train.shape, y_train.shape)
print('Test shape: ', x_test.shape, y_test.shape)

In [ ]:
print('Train shape: ', x_train.shape, y_train.shape)
print('Test shape: ', x_test.shape, y_test.shape)

#### Obučavanje i validacija SVM klasifikatora

In [ ]:
clf_svm = SVC(kernel='linear', probability=True) 
clf_svm.fit(x_train, y_train)
y_train_pred = clf_svm.predict(x_train)
y_test_pred = clf_svm.predict(x_test)
print("Train accuracy: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy: ", accuracy_score(y_test, y_test_pred))

#### Obučavanje i validacija KNN klasifikatora

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=10)
clf_knn = clf_knn.fit(x_train, y_train)
y_train_pred = clf_knn.predict(x_train)
y_test_pred = clf_knn.predict(x_test)
print("Train accuracy: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy: ", accuracy_score(y_test, y_test_pred))

##### Sa oba klasifikatora koja smo isprobali, uvek smo dobili odlične rezultate (99+%). Kako ovo možemo protumačiti?

Pa prosto, ispostavlja se da je HOG deskriptor vrlo dobar alat za izdvajanje osobina objekata sa slike, i izgleda da su te izdvojene osobine relevantne prilikom klasifikacije. Odnosno, te osobine na neki reprezentativan način predstavljaju određenu vrstu objekta, što mi onda lako možemo iskoristiti za automatizaciju klasifikacije nekim od algoritama mašinskog učenja. 

### Testiranje

Dakle, sada imamo obučen SVM klasifikator koji vrlo precizno može da predvidi da li je nešto automobil ili ne. Međutim, ovaj klasifikator radi isključivo na slikama koje su dimenzija kao u obučavajućem skupu (100 x 40). U testnom skupu su date slike koje su različitih dimenzija (dosta veće od 100 x 40) i na kojima se nalaze automobili. Potrebno je detektovati <b>gde</b> se nalazi automobil na slici. 

#### Primer testne slike

In [ ]:
itest = load_image('data/CarData/TestImages/test-0.pgm')
display_image(itest)

Tehnika kojom bi ovo trebalo raditi jeste tzv. klizeći/klizni prozor (eng. <i>sliding window</i>), gde bi se na slici pomerao "prozor", veličine 100 x 40 i za svaki prozor se računao HOG deskriptor i zatim klasifikovao (da li je automobil ili ne). Zatim prozor pomerimo za recimo 10 piksela u desno i uradimo istu stvar. Naravno, kada dođemo do kraja reda, vratimo sliku skroz levo na početak i pomerimo je za recimo 10 piksela na dole i tako dalje.

Ovim ćemo za sve moguće prozore (dimenzija 100 x 40 i sa pomerajima [10px, 0px], [0px, 10px]) dobiti rezultate klasifikacije.

<img src="images/test-im-2.png">

Međutim, ovde nastaje problem - relativno je izvesno da ukoliko smo unutar jednog prozora detektovali automobil, da ćemo i u nekom od susednih prozora detektovati isti taj automobil (prosto, translacijom za samo 10 piksela smo dobili vrlo sličnu sliku i samim tim vrlo sličan HOG deskriptor koji je ulaz u SVM klasifikator). Dakle, neophodno je na neki način spojiti više bounding box-ova u jedan koji će biti konačan rezultat. Treba voditi računa i o slikama koje sadrže više od jednog automobila,a koje su prisutne u testnom skupu podataka. 

Prilikom računanja ovog konačnog bounding box-a može se uzeti u obzir i koliko je SVM siguran (eng. <i>confidence score</i>) u svoju odluku za određeni ulazni podatak - ovu vrednost je moguće dobiti pozivanjem metode <i>predict_proba</i> nad SVM klasifikatorom. Ova metoda vraća verovatnoću ishoda klasifikacije za svaku od klasa. Zbog toga smo u konstruktoru SVM klasifikatora morali staviti <i>probability=True</i>.

<img src="images/test-im-2-nms.png">

Konačni rezultati se mogu proveriti tako što će se uporediti sa lokacijama automobila u <b>data/CarData/trueLocations.txt</b>. U ovoj datoteci, svaka linija počinje sa rednim brojem slike iz testnog foldera, a istinska lokacija automobila na slici je definisana koordinatama gornje leve tačke (oblika: (y,x)) bounding box-a (dato u datoteci), a širina  visina bounding box-a su uvek 100px i 40px, respektivno (kao veličina slika u trening skupu). 

Dakle, ako u datoteci imamo <i>0: (48, 26)</i>, to znači da se na slici <i>data/CarData/TestImages/test-0.pgm</i> automobil nalazi na lokaciji koja je definisana bounding box-om <i>((48, 26), (48+40, 26+100)) = ((48, 26), (88, 126))</i>. Ako se u datoteci nalazi više koordinata u jednoj liniji, to znači da na toj slici ima više automobila.

Metrika kojom se meri poklapanje istinskog bounding box-a i dobijenog bounding box-a je <i>Jaccard index</i>, odnosno <i>IoU (Intersection over Union)</i>. Dakle, za dva bounding box-a, pronaći njihov presek i uniju, a zatim prebrojati piksele u preseku i u uniji i na kraju podeliti ova dva broja:

$$IoU=\frac{|A \cap B|}{|A \cup B|}$$

<img src="images/iou_viz.png">

Ukoliko je poklapanje potpuno, vrednost <i>IoU</i> će biti 1, ako poklapanja nema biće 0, u slučaju delimičnog poklapanja biće neda vrednost između 0 i 1. Vrednosti veće od 0.5 (tj. **IoU > 0.5**) se generalno smatraju "dobrom" predikcijom.
<img src="images/iou_examples.png">

### Primer:  

Iskoristićemo HOG i SVM za detektovanje automobila na slici iz testnog skupa podataka. 


Prvo ćemo implementirati metodu koja će računati HOG deskriptor i *confidence score* SVM klasifikatora za prosleđeni prozor.

In [ ]:
def classify_window(window):
    features = hog.compute(window).reshape(1, -1)
    return clf_svm.predict_proba(features)[0][1]

Sada implementiramo metodu koja će "obilaziti" sliku pomoću klizećeg/kliznog prozora i "pronalaziti" automobile.

In [ ]:
def process_image(image, step_size, window_size=(100, 40)):
    best_score = 0
    best_window = None
    for y in range(0, image.shape[0], step_size):
        for x in range(0, image.shape[1], step_size):
            this_window = (y, x) # zbog formata rezultata
            window = image[y:y+window_size[1], x:x+window_size[0]]
            if window.shape == (window_size[1], window_size[0]):
                score = classify_window(window)
                if score > best_score:
                    best_score = score
                    best_window = this_window
    return best_score, best_window

Ostalo je još da implementiramo metodu koja će računati *Jaccard index* za dva prosleđena bounding box-a.

In [ ]:
def jaccard_index(true_box, predicted_box):
    y_a = max(true_box[0], predicted_box[0])
    x_a = max(true_box[1], predicted_box[1])
    y_b = min(true_box[2], predicted_box[2])
    x_b = min(true_box[3], predicted_box[3])
    
    inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)
    
    true_area = (true_box[3] - true_box[1] + 1) * (true_box[2] - true_box[0] + 1)

    pred_area = (predicted_box[3] - predicted_box[1] + 1) * (predicted_box[2] - predicted_box[0] + 1)
    
    iou = inter_area / float(true_area + pred_area - inter_area)
    
    return max(iou, 0)

Sada ćemo iskombinovati implementirane metode kako bismo videli ostvareni rezultat.

In [ ]:
score, score_window = process_image(itest, step_size=10)

In [ ]:
print(score)
print(score_window)

In [ ]:
jaccard_index([48, 26, 88, 126], [50, 20, 90, 120])

## Zadaci

### Zadatak 1 

Proveriti dobijene rezultate za sve slike iz testnog skupa sa tačnim lokacijama automobila.  
Izračunati koliki procenat automobila iz testnog skupa podataka je uspešno detektovan.

Za slike sa više automobila detaljnije istražiti [**Non-Maximum Suppresion**](https://pyimagesearch.com/2014/11/17/non-maximum-suppression-object-detection-python/) kada se radi detekcija više objekata.

### Zadatak 2 - Konji

U <b>data/HorsesData/</b> se nalazi modifikovan <b>INRIA horse dataset</b> za detekciju konja na slici. Sve slike su u boji, i različitih su dimenzija.

Skup podataka je modifikovan tako da:

* folder <b>data/HorsesData/neg</b> sadrži 170 slika negativnog uzorka (nije konj).
* folder <b>data/HorsesData/pos</b> sadrži 120 slika pozitivnog uzorka (jeste konj) sa pripadajućim <b>groundtruth</b> datotekama sa tačnim bounding box-om konja na slici. 
* folder <b>data/HorsesData/test</b> sadrži 50 testnih slika sa konjima i pripadajućim <b>groundtruth</b> datotekama.

<b>Groundtruth</b> datoteke su imenovane po šablonu: <i><b>imeSlike</b>__entires.groundtruth</i>.  
Koordinate rezultujućeg bounding box-a u <b>groundtruth</b> datotekama su date u sledećem formatu:

<i>top_left_x  top_left_y  bottom_right_x  bottom_right_y</i>

Prilikom testiranja, za poređenje bounding box-ova koristiti <i>Jaccard index</i>. 